# Converting MICrONS to NWB

This notebook converts the 2p data from [the MICrONS dataset](https://www.microns-explorer.org/) to NWB.

## Setup
The notebook needs to be run on https://codebook.datajoint.io/ using the "IARPA MICrONS Program" Sever Option


## Progress
- [x] Eye tracking
    - [x] Minor and major radius
    - [x] Eye position (x, y)
- [x] Treadmill velocity
- [x] Trials
    - [x] Start and stop times
    - [x] Stimulus type
    - [x] Condition hash
- [x] ROI masks
- [x] Fluorescence traces
- [x] Summary images
- [x] Microscope metadata
- [ ] Raw 2p data
- [x] Stimulus movie (need more RAM)
- [ ] Mapping to EM data
- [ ] Subject and session metadata
    - [ ] datetime of session
    - [ ] age, sex, and ID of mouse

In [ ]:
!pip install pynwb

In [ ]:
from microns_to_nwb.convert_microns_to_nwb import build_nwb
from pynwb import NWBHDF5IO

scan_key = {'session': 4, 'scan_idx': 7}

nwbfile = build_nwb(scan_key)


In [ ]:
subject_id = "17797"
nwbfile_path = f"{subject_id}_{scan_key['session']}_{scan_key['scan_idx']}.nwb"
with NWBHDF5IO(nwbfile_path, "w") as io:
    io.write(nwbfile)